In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
french_stopwords = list(fr_stop)
from scipy import stats

# Count words
Fonction afin de compter les mots uniques dans une liste de documents pondérés

In [71]:
doca = "je vais au tennis ce  matin a 14h"
docb = "je vais à la cantine à midi"
DocsWeights = [(doca,4),(docb,1)]

In [72]:
def ExpandWeightedDocs(DocsWeights):
    
    L = []
    for d,w in DocsWeights:
        L = L + [d]*w
    
    return L

In [73]:
def Tokenizer(randomstring,french_stopwords=french_stopwords):
    randomstring = " ".join(randomstring.split())
    words = randomstring.split(" ")
    words = [item for item in words if item not in french_stopwords]
    return words

In [74]:
def SentenceCompteur(compteur,words,weight=1):
    for w in words:
        compteur[w] = compteur.get(w,0) + weight
    return compteur

In [75]:
def ExtractCompteur(DocsWeights):
    compteur = {}
    for doc,coefficient in DocsWeights:
        sentence = Tokenizer(doc)
        compteur = SentenceCompteur(compteur,sentence,coefficient)
    return compteur

In [76]:
compteur = ExtractCompteur(DocsWeights)

# Define Time Mark

In [80]:
def DefineTimeMarks(lastdate,firstdate,step,window):

    mark0 = firstdate + window
    start = mark0
    L = [start]

    while(True):
        newmark = start+step
        if newmark>lastdate:
            break
        else:
            L.append(newmark)
            start=newmark

    return L

In [81]:
lastdate = 1000
firstdate = 587
step = 50
window = 200

# Clean RefRT

In [ ]:
LastTimeMarkTreated = 14555
LastTimeMarkTreated - window

## Tweets of Authors

In [ ]:
randomtm = TimeMarksList[-1]
TweetsAuthorsDF = BuildTweetsAuthorFromTimeMark(randomtm,WindowSize,StepSize,rtdf)

# ALL IN ALL

In [ ]:
# LOAD DATA
# ...
OldListOfTMResults = PickleLoad(os.path.join(Root,FolderProject,"SequenceOfGraphResults.pkl"))

In [ ]:
# DEFINE PREREQUISITE VARIABLES
firstdate,lastdate = DefineFirstLastDate(rtdf,StepSize)
ListOfTM = DefineTimeMarks(lastdate,firstdate,StepSize,WindowSize)

In [ ]:
# ANALYTICS
ListOfTMResults = ListOfTMTreatment(ListOfTM,WindowSize,StepSize,rtdf)
LogTMAddSize = len(ListOfTMResults)
LogBasicStats = [(len(item["Links"]),len(item["Tweets"])) for item in ListOfTMResults]
LogBasicStats = np.array(LogBasicStats)
LogBasicStats = stats.describe(LogBasicStats)
LogBasicStats = dict(LogBasicStats._asdict())

ListOfTMResults = ListOfTMResults + OldListOfTMResults
LogTMNowSize = len(ListOfTMResults)
LogBasicStats2 = [(len(item["Links"]),len(item["Tweets"])) for item in ListOfTMResults]
LogBasicStats2 = np.array(LogBasicStats2)
LogBasicStats2 = stats.describe(LogBasicStats2)
LogBasicStats2 = dict(LogBasicStats2._asdict())

In [ ]:
# SAVE RESULTS
PickleDump(os.path.join(Root,FolderProject,"SequenceOfGraphResults.pkl"),ListOfTMResults)

In [ ]:
# SAVE LOGS
mylog = {"TMAddSize":LogTMAddSize,
         "TMNowSize":LogTMNowSize,
         "AddBasicStats":LogBasicStats,
         "NewBasicStats":LogBasicStats2}
WriteLogs(os.path.join(Root,FolderProject,"Graph.log"),mylog)

# Functions

In [ ]:
def DefineFirstLastDate(rtdf,StepSize):

    lastdate = rtdf.TWEETUNIXEPOCH.max()

    if len(OldListOfTMResults) == 0:
        firstdate = rtdf.TWEETUNIXEPOCH.min()
    else:
        LastTimeMark = max([item["TimeMark"] for item in OldListOfTMResults])
        firstdate = LastTimeMark + StepSize

    return firstdate,lastdate

In [ ]:
def ListOfTMTreatment(ListOfTM,WindowSize,StepSize,rtdf):

    L = []
    for randomtm in ListOfTM:
        TimeMarkResultsDic = {}
        LinksDic = BuildLinksDocFromTimeMark(randomtm,WindowSize,StepSize,rtdf)
        TweetsAuthorsDF = BuildTweetsAuthorFromTimeMark(randomtm,WindowSize,StepSize,rtdf)
        TimeMarkResultsDic[randomtm] = {"Links":LinksDic,
                                        "Tweets":TweetsAuthorsDF,
                                        "TimeMark":randomtm,
                                        "LinksWindow":randomtm - WindowSize,
                                        "TweetsWindow":randomtm - StepSize}
        L.append(TimeMarkResultsDic)
    return L

In [ ]:
def BuildTweetsAuthorFromTimeMark(randomtm,WindowSize,StepSize,rtdf):
    
    periods = ExtractTheTwoPeriods(randomtm,WindowSize,StepSize)
    debut,fin = periods["TweetWindow"]
    myfilter = (rtdf.TWEETUNIXEPOCH>=debut) & (rtdf.TWEETUNIXEPOCH<=fin)
    temprtdf = rtdf.copy()[myfilter]
    temprtdf.reset_index(drop=True,inplace=True)
    
    # Modifier authoridname et tweetidname
    #TweetsAuthorsDF = temprtdf.groupby(authoridname)[tweetidname].apply(list).reset_index()
    TweetsAuthorsDF = temprtdf.groupby(["AUTHORID","AUTHORTWEETID"]).size().reset_index()
    
    
    return TweetsAuthorsDF

# Clean RTDF, another notebook

1. On charge la liste de results
2. On trouve la derniere fenetre
3. On extrait la borne supérieure de la fenetre
4. On supprime tout les rt dont la date d'émission est inférieur à la borne supérieur

In [ ]:
#Load data
# ... rtdf
ListOfTMResults = PickleLoad(os.path.join(Root,FolderProject,"SequenceOfGraphResults.pkl"))
LastTimestamp = max([item["TimeMark"] for item in ListOfTMResults ])
FirstTimestamp = min([item["TimeMark"] for item in ListOfTMResults ])
CleanRTDF = rtdf.copy()[rtdf.TWEETUNIXEPOCH>LastTimestamp]
PickleDump(os.path.join(Root,FolderProject,"rtdf.pkl"),CleanRTDF)

In [ ]:
# Nombre de lignes dans rtdf avant / apres
LogNoldRTDF = len(rtdf)
LogNnewRTDF = len(CleanRTDF)

LogFirstDateOld = str(pd.to_datetime(rtdf.TWEETUNIXEPOCH.min(),unit="s"))
LogLastDateOld = str(pd.to_datetime(rtdf.TWEETUNIXEPOCH.max(),unit="s"))

LogFirstDateNew = str(pd.to_datetime(CleanRTDF.TWEETUNIXEPOCH.min(),unit="s"))
LogLastDateNew = str(pd.to_datetime(CleanRTDF.TWEETUNIXEPOCH.max(),unit="s"))

log ={"NoldRTDF":LogNoldRTDF,
      "NnewRTDF":LogNnewRTDF,
      "FirstDateOld":LogFirstDateOld,
      "LastDateOld":LogLastDateOld,
      "FirstDateNew":LogFirstDateNew,
      "LastDateNew":LogLastDateNew}

WriteLogs(os.path.join(Root,FolderProject,"Clean.log"),log)

# Counting Words

Pour chaque cluster : 
 * Sélection des lignes appartenant à un cluster
 * On extrait tout les tweets ids des nodes
 * 